In [4]:
from __future__ import print_function

import os
import sys

import numpy as np
import tensorflow as tf

sys.path.extend([".", ".."])  # adds the folder from which you call the script
os.environ["CASAPOSE_INFERENCE"] = "True"

from casapose.data_handler.image_only_dataset import ImageOnlyDataset
from casapose.data_handler.vectorfield_dataset import VectorfieldDataset
from casapose.pose_estimation.pose_evaluation import poses_pnp
from casapose.pose_estimation.voting_layers_2d import CoordLSVotingWeighted
from casapose.pose_models.tfkeras import Classifiers
from casapose.utils.config_parser import parse_config

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

In [21]:
#opt = parse_config()

class opt:
    modelname = 'casapose_c_gcu5'
    estimate_confidence = 1
    estimate_coords = 1
    confidence_regularization = 1
    object = 'obj_000001,obj_000005,obj_000006,obj_000008,obj_000009,obj_000010,obj_000011,obj_000016'

    no_points = 9
    save_debug_batch = 0

    imagesize = (448, 448)
    imagesize_test = (480, 640)
    crop_factor = 0.933333333
    pretrained = 1
    manualseed = 1237

    # losses
    mask_loss_weight = 1.0
    vertex_loss_weight = 0.5
    proxy_loss_weight = 0.015
    keypoint_loss_weight = 0.007
    filter_vertex_with_segmentation = 1
    filter_high_proxy_errors = 0
    use_bpnp_reprojection_loss = 0
    max_keypoint_pixel_error = 12.5

    # learning rate
    lr = 0.001
    lr_decay = 0.5
    lr_epochs_steps = 50,75,90

    # general
    gpuids = 0,1
    loginterval = 10
    epochs = 100
    batchsize = 4
    saveinterval = 5
    validationinterval = 1

    # data preprocessing
    workers = 0
    prefetch = 10

    # augmentation
    translation = 0
    rotation = 0
    noise = 0.0001
    brightness = 0.001
    contrast = 0.001
    saturation = 0.001
    hue = 0.001
    use_imgaug = 1

    # test
    min_object_size_test = 200
    write_poses = 0
    save_eval_batches = 0

    # output
    net = 'training_checkpoints'
    outf = 'train_casapose_8_16_objects'

    # config
    train_vectors_with_ground_truth = 1
    load_h5_weights = 0
    copy_weights_from_backup_network = 0
    copy_weights_add_confidence_maps = 0
    objects_in_input_network = 8
    objects_to_copy = 1
    objects_to_copy_list = 'config/objects_to_copy.csv'
    
    confidence_filter_estimates = 1
    confidence_choose_second = 0
    train_vectors_with_ground_truth = 0
    datatest_wxyz_quaterion = 0
    filter_test_with_gt = 0
    
    evalf = 'output'
    datatest = 'import_data/test/test'
    datameshes = 'import_data/test/models'
    #datatest = '/workspace/CASAPose/import_data/test/test'
    #datameshes = '/workspace/CASAPose/import_data/test/models'
    data = ''
    datatest_path_filter = None
    color_dataset = 1
    train_validation_split = None #0.9
    backbonename = 'resnet18'
    load_h5_weights = 1
    #load_h5_filename = '../../../data/pretrained_models/result_w'
    load_h5_filename = 'data/pretrained_models' + '/result_w.h5'

In [7]:
if not os.path.exists(opt.evalf):
    os.makedirs(opt.evalf)

checkpoint_path = opt.outf + "/" + opt.net

frozen_path = opt.outf + "/frozen_model"
img_out_path = opt.outf + "/control_output"

In [9]:
def create_dir(path):
    try:
        os.makedirs(path)
    except OSError:
        pass
    
create_dir(img_out_path)

In [ ]:
# set the manual seed.
np.random.seed(opt.manualseed)
tf.random.set_seed(opt.manualseed)

In [11]:
test_dataset = None

device_ids = []
if len(opt.gpuids) == 1 and opt.gpuids[0] < 0:
    device_ids.append("/cpu:0")
else:
    device_ids.append("/gpu:{}".format(opt.gpuids[0]))
print(device_ids)

objectsofinterest = [x.strip() for x in opt.object.split(",")]
no_objects = len(objectsofinterest)
separated_vectorfields = opt.modelname == "pvnet"


testingdata = None
normal_imgs = [0.5, 0.5]

use_split = False
if opt.data == opt.datatest:
    print("split datasets with ratio {}".format(opt.train_validation_split))
    use_split = True

test_batches = 0

['/gpu:0']


In [ ]:
test_dataset = VectorfieldDataset(
    root="import_data/test/test", # Hardcoded param
    path_meshes="import_data/test/models", # Hardcoded param
    path_filter_root=None, # Hardcoded param
    color_input=True, # Hardcoded param
    no_points=9, # Taken from config_8.ini
    objectsofinterest=objectsofinterest,
    noise=0.00001,
    data_size=None,
    save=False, # Hardcoded param
    normal=normal_imgs,
    contrast=0.00001,
    brightness=0.00001,
    hue=0.00001,
    saturation=0.00001,
    random_translation=(0, 0),
    random_rotation=0,
    random_crop=False,
    use_validation_split=use_split,
    use_train_split=False,
    train_validation_split=None,
    output_folder=None,
    visibility_filter=False,
    separated_vectorfields=(opt.modelname == "pvnet"),
)

In [22]:
test_dataset = ImageOnlyDataset(root=opt.datatest)
testing_images, _ = test_dataset.generate_dataset(1)

test_dataset = VectorfieldDataset(
    root=opt.datatest,
    path_meshes=opt.datameshes,
    path_filter_root=opt.datatest_path_filter,
    color_input=opt.color_dataset,
    no_points=opt.no_points,
    objectsofinterest=objectsofinterest,
    noise=0.00001,
    data_size=None,
    save=opt.save_debug_batch,
    normal=normal_imgs,
    contrast=0.00001,
    brightness=0.00001,
    hue=0.00001,
    saturation=0.00001,
    random_translation=(0, 0),
    random_rotation=0,
    random_crop=False,
    use_validation_split=use_split,
    use_train_split=False,
    train_validation_split=opt.train_validation_split,
    output_folder=opt.evalf,
    visibility_filter=False,
    separated_vectorfields=(opt.modelname == "pvnet"),
)
print(len(test_dataset))
testingdata, test_batches = test_dataset.generate_dataset(
    1, 1, 0, opt.imagesize_test, 1.0, 1, no_objects, shuffle=False
)

mesh_vertex_array, mesh_vertex_count = test_dataset.generate_object_vertex_array()

print("testing data: {} batches".format(test_batches))


import_data/test/models/obj_000001/obj_000001.ply
import_data/test/models/obj_000002/obj_000002.ply
import_data/test/models/obj_000003/obj_000003.ply
import_data/test/models/obj_000004/obj_000004.ply
import_data/test/models/obj_000005/obj_000005.ply
import_data/test/models/obj_000006/obj_000006.ply
import_data/test/models/obj_000007/obj_000007.ply
import_data/test/models/obj_000008/obj_000008.ply
import_data/test/models/obj_000009/obj_000009.ply
import_data/test/models/obj_000010/obj_000010.ply
import_data/test/models/obj_000011/obj_000011.ply
import_data/test/models/obj_000012/obj_000012.ply
import_data/test/models/obj_000013/obj_000013.ply
import_data/test/models/obj_000014/obj_000014.ply
import_data/test/models/obj_000015/obj_000015.ply
import_data/test/models/obj_000016/obj_000016.ply
import_data/test/test/.DS_Store
import_data/test/test/000001
5
testing data: 5.0 batches


In [26]:
input_segmentation_shape = None
if opt.train_vectors_with_ground_truth is True:
    input_segmentation_shape = (
        opt.imagesize_test[0],
        opt.imagesize_test[1],
        1 + no_objects,
    )


height = opt.imagesize_test[0]
width = opt.imagesize_test[1]

CASAPose = Classifiers.get(opt.modelname)
ver_dim = opt.no_points * 2
if opt.modelname == "pvnet":
    ver_dim = ver_dim * no_objects

if opt.estimate_confidence:
    assert separated_vectorfields is not None, "confidence not compaitble with this model"
    ver_dim += opt.no_points

In [29]:
net = CASAPose(
    ver_dim=ver_dim,
    seg_dim=1 + no_objects,
    input_shape=(height, width, 3),
    input_segmentation_shape=input_segmentation_shape,
    weights="imagenet",
    base_model=opt.backbonename,
)

checkpoint_prefix = os.path.join(checkpoint_path, "ckpt")
checkpoint = tf.train.Checkpoint(network=net)  # , optimizer=optimizer)

if opt.load_h5_weights is True:
    net.load_weights(frozen_path + "/" + opt.load_h5_filename + ".h5", by_name=True, skip_mismatch=True)

elif opt.net != "":
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path)).expect_partial()

for layer in net.layers:
    layer.trainable = False

net.summary()

[<KerasTensor: shape=(None, 240, 320, 64) dtype=float32 (created by layer 'relu0')>, <KerasTensor: shape=(None, 120, 160, 64) dtype=float32 (created by layer 'stage2_unit1_relu1')>, <KerasTensor: shape=(None, 60, 80, 128) dtype=float32 (created by layer 'stage3_unit1_relu1')>, <KerasTensor: shape=(None, 60, 80, 256) dtype=float32 (created by layer 'stage4_unit1_relu1')>, <KerasTensor: shape=(None, 60, 80, 512) dtype=float32 (created by layer 'relu1')>]
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 480, 640, 3  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)            

                                8)                                'model[0][0]']                  
                                                                                                  
 zero_padding2d_21 (ZeroPadding  (None, 242, 322, 12  0          ['concatenate_2[0][0]']          
 2D)                            8)                                                                
                                                                                                  
 pv_block_4_conv2d (Conv2D)     (None, 240, 320, 32  36864       ['zero_padding2d_21[0][0]']      
                                )                                                                 
                                                                                                  
 pv_block_4_bn (SyncBatchNormal  (None, 240, 320, 32  128        ['pv_block_4_conv2d[0][0]']      
 ization)                       )                                                                 
          

 pv_block_7_prepare_conv2d (Par  (None, 60, 80, 128)  442368     ['concatenate_4[0][0]',          
 tialConvolution)                                                 'segmentation_eighth_size[0][0]'
                                                                 ]                                
                                                                                                  
 pv_block_7_clade (ClassAdaptiv  (None, 60, 80, 128)  2560       ['pv_block_7_prepare_conv2d[0][0]
 eWeightedNormalization)                                         ',                               
                                                                  'segmentation_eighth_size[0][0]'
                                                                 ]                                
                                                                                                  
 tf.math.multiply_5 (TFOpLambda  (None, 60, 80, 128)  0          ['pv_block_7_clade[0][0]']       
 )        

                                                                                                  
 pv_block_10_prepare_conv2d (Pa  (None, 480, 640, 32  10080      ['concatenate_7[0][0]',          
 rtialConvolution)              )                                 'tf.stop_gradient[0][0]']       
                                                                                                  
 pv_block_10_clade (ClassAdapti  (None, 480, 640, 32  640        ['pv_block_10_prepare_conv2d[0][0
 veWeightedNormalization)       )                                ]',                              
                                                                  'tf.stop_gradient[0][0]']       
                                                                                                  
 tf.math.multiply_8 (TFOpLambda  (None, 480, 640, 32  0          ['pv_block_10_clade[0][0]']      
 )                              )                                                                 
          

In [306]:
img_batch[4].shape

TensorShape([1, 8, 1, 9, 3])

In [311]:
def runnetwork(loader_iterator, batches, no_objects):
    
    #@tf.function
    def test_step(img_batch):
        start_time = tf.timestamp()
        
        keypoints = img_batch[4]
        camera_matrix = img_batch[5]
            
        no_points = opt.no_points
        net_input = [tf.expand_dims(img_batch[0][0], 0)]
        output_net = net(net_input, training=False)  # all stages are present here
        output_seg, output_dirs, confidence = tf.split(output_net, [no_objects, no_points * 2, -1], 3)
        coordLSV_in = [output_seg, output_dirs, confidence]
        coords = CoordLSVotingWeighted(
            name="coords_ls_voting",
            num_classes=no_objects,
            num_points=no_points,
            filter_estimates=True,
        )(coordLSV_in)
        poses_est = poses_pnp(
            coords, output_seg, keypoints, camera_matrix, no_objects - 1, min_num=opt.min_object_size_test
        )
        tf.print(poses_est[0, 0, 0, 0, 0])  # access memory to get sure everything is done

        end_time = tf.timestamp()
        time_needed = end_time - start_time
        return time_needed, poses_est, coords

    speed = []
    img_batches = []
    est_poses = []
    coords_list = []
    
    for batch_idx in range(batches):
        img_batch = loader_iterator.get_next()
                
        time_needed, poses_est, coords = test_step(img_batch)
        speed.append(time_needed)
        with open(opt.evalf + "/speed_eval.csv", "a") as file:
            s = "{},{:.7f}\n".format(batch_idx + 1, time_needed)
            file.write(s)
            
        img_batches.append(img_batch)
        est_poses.append(poses_est)
        coords_list.append(coords)

    tf.print("average speed: {}".format(tf.reduce_mean(speed[10:])), summarize=-1)

    return loader_iterator, img_batches, est_poses, coords_list

In [312]:
print("Test Batches: {} ".format(test_batches))

testingdata_iterator = iter(testingdata)
img_batch = testingdata_iterator.get_next()

keypoints = img_batch[4]
camera_matrix = img_batch[5]

type(img_batch)

Test Batches: 5.0 


tuple

In [313]:
len(img_batch)

13

In [ ]:
#testingdata_iterator = iter(testing_images)

testingdata_iterator = iter(testingdata)

loader_iterator, img_batches, est_poses, coords_list = runnetwork(testingdata_iterator, 
                                                                  int(test_batches),
                                                                  no_objects + 1)

-0.397043586


In [125]:
from casapose.utils.image_utils import get_all_vectorfields
from casapose.utils.loss_functions import (
    keypoint_reprojection_loss,
    proxy_voting_dist,
    proxy_voting_loss_v2,
    smooth_l1_loss,
)
from casapose.pose_estimation.pose_evaluation import (
    estimate_and_evaluate_poses,
    evaluate_pose_estimates,
)


def compute_loss(
    output_seg,
    target_seg,
    output_vertex,
    target_vertex,
    target_points,
    mask_loss_weight=1.0,
    vertex_loss_weight=1.0,
    proxy_loss_weight=1.0,
    kp_loss_weight=0.0,
    kp_loss=None,
):

    oc = np.int32(target_seg.shape[3] - 1)  # object count
    vc = target_points.shape[3] * 2  # vertex count
    kp_loss = tf.constant(0.0, dtype=tf.float32)
    mask_loss = tf.reduce_mean(
        input_tensor=tf.nn.softmax_cross_entropy_with_logits(labels=target_seg, logits=output_seg)
    )
    separated_vectors = oc > 1 and output_vertex.shape[-1] == (oc * vc)  # original pvnet with multiple objects

    if kp_loss is None:
        kp_loss = tf.constant(0.0, dtype=tf.float32)

    if separated_vectors:  #
        vertex_loss = sum(
            smooth_l1_loss(
                output_vertex[:, :, :, i * vc : (i + 1) * vc],
                target_vertex[:, :, :, i * vc : (i + 1) * vc],
                target_seg[:, :, :, i + 1 : i + 2],
            )
            for i in range(oc)
        )
        proxy_loss = sum(
            proxy_voting_loss_v2(
                output_vertex[:, :, :, i * vc : (i + 1) * vc],
                target_points[:, i : i + 1, :, :, :],
                vertex_one_hot_weights=target_seg[:, :, :, i + 1 : i + 2],
                vertex_weights=target_seg[:, :, :, i + 1 : i + 2],
            )
            for i in range(oc)
        )
    else:
        vertex_loss = smooth_l1_loss(
            output_vertex,
            target_vertex,
            target_seg[:, :, :, 0:1],
            invert_weights=True,
        )
        proxy_loss = proxy_voting_loss_v2(
            output_vertex,
            target_points,
            vertex_one_hot_weights=target_seg[:, :, :, 1:],
            vertex_weights=target_seg[:, :, :, 0:1],
            invert_weights=True,
            loss_per_object=False,
        )

    loss = (
        tf.multiply(mask_loss, mask_loss_weight)
        + tf.multiply(proxy_loss, proxy_loss_weight)
        + tf.multiply(vertex_loss, vertex_loss_weight)
        + tf.multiply(kp_loss, kp_loss_weight)
    )
    return [loss, mask_loss, vertex_loss, proxy_loss, kp_loss]

In [193]:
from casapose.utils.inf_dataset_utils import save_eval_batch

def runnetwork(loader_iterator, batches):
    
    images = []
    stats = []
    est_poses = []
    image_batches = []
    
    for batch_idx in range(batches):

        img_batch = loader_iterator.get_next()

        loss = 0

        target_seg = img_batch[1]
        target_vertex = img_batch[3]
        keypoints = img_batch[4]
        cam_mat = img_batch[5]
        diameters = img_batch[6]
        offsets = img_batch[7]
        filtered_seg = img_batch[8]
        poses_gt = img_batch[10]
        confidence = None
        kp_loss = None
        no_features = target_vertex.shape[3]  # vertex count
        no_points = opt.no_points
        no_objects = target_seg.shape[3]

        separated_vectorfields = opt.modelname == "pvnet"

        target_dirs = get_all_vectorfields(
            target_seg,
            target_vertex,
            filtered_seg,
            separated_vectorfields,
        )

        net_input = [img_batch[0]]
        output_net = net(net_input, training=False)  # all stages are present here

        if opt.estimate_confidence:
            output_seg, output_dirs, confidence = tf.split(output_net, [no_objects, no_points * 2, -1], 3)
        else:
            output_seg, output_dirs = tf.split(output_net, [no_objects, -1], 3)

        if opt.estimate_coords:
            if opt.train_vectors_with_ground_truth:
                coordLSV_in = [target_seg, output_dirs, confidence]
            else:
                coordLSV_in = [output_seg, output_dirs, confidence]

            coords = CoordLSVotingWeighted(
                name="coords_ls_voting",
                num_classes=no_objects,
                num_points=no_points,
                filter_estimates=opt.confidence_filter_estimates,
                output_second_largest_component=opt.confidence_choose_second,
            )(coordLSV_in)

            kp_loss, poses_est, points_est = keypoint_reprojection_loss(
                coords, 
                output_seg,
                poses_gt,
                keypoints,
                target_seg,
                cam_mat,
                offsets,
                confidence,
                min_num=opt.min_object_size_test,
                min_num_gt=1,
                use_bpnp_reprojection_loss=opt.use_bpnp_reprojection_loss,
                estimate_poses=True,
                filter_with_gt=opt.filter_test_with_gt,
            )

        if opt.estimate_coords:
            pose_stats, estimated_poses, estimated_points = evaluate_pose_estimates(
                points_est,
                poses_est,
                poses_gt,
                target_seg,
                keypoints,
                cam_mat,
                diameters,
                evaluation_points=mesh_vertex_array,
                object_points_3d_count=mesh_vertex_count,
                min_num=1,
            )
            estimated_poses = tf.squeeze(estimated_poses, axis=2)
        else:
            pose_stats, estimated_poses, estimated_points = estimate_and_evaluate_poses(
                output_seg,
                target_seg,
                output_dirs,
                poses_gt,
                keypoints,
                cam_mat,
                diameters,
                offsets,
                evaluation_points=mesh_vertex_array,
                object_points_3d_count=mesh_vertex_count,
                min_num=1,
            )

        # end_time = tf.timestamp()
        # time_needed = end_time - start_time

        loss = compute_loss(
            output_seg,
            target_seg,
            output_dirs,
            target_dirs,
            target_vertex,
            opt.mask_loss_weight,
            opt.vertex_loss_weight,
            opt.proxy_loss_weight,
            opt.keypoint_loss_weight,
            kp_loss=kp_loss,
        )
        loss.append(pose_stats)

        _, object_loss_values = proxy_voting_dist(
            output_dirs,
            target_vertex,
            vertex_one_hot_weights=target_seg[:, :, :, 1:],
            vertex_weights=target_seg[:, :, :, 0:1],
            invert_weights=True,
        )
        loss.append(object_loss_values)

        beta = tf.cast(1e6, dtype=output_seg.dtype)
        hot_seg = tf.expand_dims(tf.expand_dims(tf.nn.softmax(output_seg * beta), -1), -1)[:, :, :, 1:, :, :]
        w = tf.math.softplus(confidence)
        hot_seg_ = tf.squeeze(hot_seg, axis=-1)
        w_ = hot_seg_ * tf.expand_dims(w, axis=-2)
        w_max = tf.reduce_max(w_, axis=[0, 1, 2], keepdims=True)
        w_min = tf.reduce_min(w_, axis=[0, 1, 2], keepdims=True)
        w_ = tf.math.divide_no_nan(tf.subtract(w_, w_min), tf.subtract(w_max, w_min))
        confidence = tf.reduce_sum(w_, axis=-2)

        add_correct = loss[5][1]
        
        image = save_eval_batch(
                img_batch,
                output_seg,
                target_dirs,
                output_dirs,
                estimated_poses,
                estimated_points,
                no_objects - 1,
                no_features,
                # path_out=opt.evalf + "/visual_batch_eval_mask",
                confidence=confidence,
                add_correct=add_correct,
                )
            
        #tf.print("Estimated Coordinates: {}".format(estimated_points), summarize=-1)
        #tf.print("Estimated Poses: {}".format(estimated_poses), summarize=-1)
        #tf.print("Segmentation Masks: {}".format(output_seg), summarize=-1)
        #tf.print("Image: {}".format(type(image)), summarize=-1)
        #tf.print("Labels: TESTING") #{}".format(test_pose_count_gt), summarize=-1)
        tf.print("Objects of Interest: {}".format(objectsofinterest), summarize=-1)
        tf.print("length: {}".format(len(objectsofinterest)), summarize=-1)
        tf.print("Number of Objects: {}".format(no_objects), summarize=-1)
        

        images.append(image)
        stats.append(pose_stats)
        est_poses.append(estimated_poses)
        image_batches.append(img_batch)
    
    return images, stats, est_poses, image_batches
        

        #return image, pose_stats, estimated_poses, output_seg, loss
        # return loss, estimated_points, estimated_poses, output_seg
        # return estimated_points, estimated_poses, output_seg


In [194]:
print("Test Batches: {} ".format(test_batches))

testingdata_iterator = iter(testingdata)
images, stats, est_poses, image_batches = runnetwork(testingdata_iterator, int(test_batches))

Test Batches: 5.0 
[TensorShape([8, 9, 2]), TensorShape([8, 9, 3]), TensorShape([3, 3])]
[0 0 0 0 0 0 0 0]
Objects of Interest: ['obj_000001', 'obj_000005', 'obj_000006', 'obj_000008', 'obj_000009', 'obj_000010', 'obj_000011', 'obj_000016']
length: 8
Number of Objects: 9
[TensorShape([8, 9, 2]), TensorShape([8, 9, 3]), TensorShape([3, 3])]
[0 0 0 0 0 0 0 0]
Objects of Interest: ['obj_000001', 'obj_000005', 'obj_000006', 'obj_000008', 'obj_000009', 'obj_000010', 'obj_000011', 'obj_000016']
length: 8
Number of Objects: 9
[TensorShape([8, 9, 2]), TensorShape([8, 9, 3]), TensorShape([3, 3])]
[0 0 0 0 0 0 0 0]
Objects of Interest: ['obj_000001', 'obj_000005', 'obj_000006', 'obj_000008', 'obj_000009', 'obj_000010', 'obj_000011', 'obj_000016']
length: 8
Number of Objects: 9
[TensorShape([8, 9, 2]), TensorShape([8, 9, 3]), TensorShape([3, 3])]
[0 0 0 0 0 0 0 0]
Objects of Interest: ['obj_000001', 'obj_000005', 'obj_000006', 'obj_000008', 'obj_000009', 'obj_000010', 'obj_000011', 'obj_000016']


In [195]:
est_poses[0].shape

TensorShape([1, 8, 3, 4])

In [296]:
from casapose.utils.geometry_utils import apply_offsets, project
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
from casapose.utils.draw_utils import draw_bb

import cv2
import matplotlib as plt
import numpy as np
import tensorflow as tf
from PIL import Image

def save_poses_single_sample(
    img,
    estimated_poses,
    gt_pose,
    offsets,
    cuboids,
    keypoints,
    camera_matrix,
    path,
    file_prefix,
    normal=[0.5, 0.5],
):
        
    img_keypoints = tf.cast(((img * normal[1]) + normal[0]) * 255, dtype=tf.uint8).numpy()
    img_cuboids = img_keypoints.copy()

    eps = 1e-4
    
    for obj_idx, obj_pose in enumerate(estimated_poses):
        
        print(obj_idx)

        inst_idx = 0
        obj_pose_est = estimated_poses[obj_idx].numpy()
        instance_cuboids = cuboids[obj_idx][inst_idx].numpy()
        instance_keypoints = keypoints[obj_idx][inst_idx].numpy()     


        valid_est = np.abs(np.sum(obj_pose_est)) > eps
        # draw bb - ESTIMATED
        if valid_est: 
            print('est')
            print(obj_pose_est.shape)
            transformed_cuboid_points2d, _ = project(instance_cuboids, camera_matrix.numpy(), obj_pose_est)   
            
            #transformed_cuboid_points2d = apply_offsets(transformed_cuboid_points2d, offsets)
            #transformed_cuboid_points2d = transformed_cuboid_points2d.numpy()
            #transformed_cuboid_points2d = np.reshape(transformed_cuboid_points2d, (8,2))
            
            print(transformed_cuboid_points2d.shape)
            draw_bb(transformed_cuboid_points2d, img_cuboids, (0, 255, 0))
        else: 
            print('skipped obj est')

        # GT
        if gt_pose is not None: 
            instance_pose_gt = gt_poses[obj_idx][inst_idx].numpy()
            valid_gt = np.abs(np.sum(instance_pose_gt)) > eps
            
            if valid_gt: 
                print('gt')
                gt_pose_est = gt_pose[obj_idx].numpy()
                print(gt_pose_est[0].shape)
                transformed_cuboid_points2d_gt, _ = project(instance_cuboids, 
                                                            camera_matrix.numpy(), 
                                                            gt_pose_est[0])   
                print(gt_pose_est[0])
                draw_bb(transformed_cuboid_points2d_gt, img_cuboids, (255, 0, 0))
            else: 
                print('skipped obj')

    # save image
    img_cuboids = Image.fromarray((img_cuboids).astype("uint8"))
    img_cuboids.save(path + "/" + str(file_prefix) + "_cuboids_all.png")

In [297]:
testingdata_iterator = iter(testingdata)
img_batch = testingdata_iterator.get_next()

In [300]:
#img_batch = img_batches[0]
poses_est = est_poses[0]
poses_est.shape

TensorShape([1, 8, 1, 3, 4])

In [301]:
poses_est = np.reshape(poses_est, (1, 8, 3, 4))
poses_est.shape
# needs to be: 
# TensorShape([1, 8, 3, 4])

TensorShape([1, 8, 3, 4])

In [302]:
img = img_batch[0][0]
keypoints = img_batch[4][0]
cam_mat = img_batch[5][0]
offsets = img_batch[7][0]
cuboids = img_batch[9][0]
gt_poses = img_batch[10][0]
estimated_poses=poses_est[0]
path_out = 'test'

In [303]:
gt_poses.shape
# TensorShape([8, 1, 3, 4])

TensorShape([8, 1, 3, 4])

In [304]:
save_poses_single_sample(
    img = img,
    estimated_poses=estimated_poses,
    gt_pose=gt_poses,
    offsets=offsets,
    cuboids=cuboids,
    keypoints=keypoints,
    camera_matrix=cam_mat,
    path=path_out,
    file_prefix=0,
    normal=[0.5, 0.5]
)

0
est
(3, 4)
(8, 2)
gt
(3, 4)
[[-8.3105749e-01 -4.9617833e-01  2.5129774e-01  1.8149728e+02]
 [-5.1251614e-01  5.0768352e-01 -6.9252050e-01  3.1408548e+02]
 [ 2.1603394e-01 -7.0431846e-01 -6.7621356e-01  1.0016328e+03]]
1
skipped obj est
gt
(3, 4)
[[ 9.4615239e-01  2.0407134e-01  2.5129774e-01 -6.1099270e+01]
 [ 3.2305470e-01 -6.4502031e-01 -6.9252038e-01  3.7482285e+02]
 [ 2.0768568e-02  7.3641270e-01 -6.7621368e-01  7.7388177e+02]]
2
est
(3, 4)
(8, 2)
gt
(3, 4)
[[ 5.2214855e-01 -8.1499100e-01  2.5129774e-01  2.2954282e+02]
 [-4.9126270e-01 -5.2827692e-01 -6.9252038e-01  7.4394020e+01]
 [ 6.9715267e-01  2.3814531e-01 -6.7621374e-01  1.2459657e+03]]
3
skipped obj est
gt
(3, 4)
[[-4.1060883e-01 -8.7649864e-01  2.5129774e-01  1.7613786e+02]
 [-7.0892721e-01  1.3355748e-01 -6.9252044e-01  1.8576587e+02]
 [ 5.7343054e-01 -4.6250683e-01 -6.7621362e-01  1.0451112e+03]]
4
est
(3, 4)
(8, 2)
gt
(3, 4)
[[ 2.2433765e-01 -9.4155300e-01  2.5129780e-01  2.7690518e+02]
 [-6.3793588e-01 -3.3682850e-01